# Criando o Modelo (Naive Bayes)

## Imports bibliotecas

In [28]:
import pandas as pd
import re
from nltk import tokenize
import nltk
from string import punctuation
from unidecode import unidecode
import unidecode

## Importando dados de treino e teste

In [29]:
df_imdb = pd.read_csv('./data/imdb-reviews-pt-br.csv')
df_imdb = df_imdb[['text_pt', 'sentiment']]
df_imdb.columns = ['text', 'sentiment']

In [30]:
df_neg = pd.read_csv('./data/negativo.csv', header=None)
df_pos = pd.read_csv('./data/positivo.csv', header=None)

df_pos['sentiment'] = 'pos'
df_neg['sentiment'] = 'neg'
df_neg.columns = ['text', 'sentiment']
df_pos.columns = ['text', 'sentiment']

In [31]:
dfs = [df_imdb, df_pos, df_neg]
dfTreino = pd.concat(dfs, ignore_index=True)

In [32]:
mapeamento = {'pos': 'Positivo', 'neg': 'Negativo'}

# Aplica o mapeamento à coluna
dfTreino['sentiment'] = dfTreino['sentiment'].map(mapeamento)

## Pré-Processamento

### Removendo hashtags e menções.

In [33]:
def limpando_chars(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # Removendo menções
    text = re.sub(r'#\w+', '', text) # Removendo #hashtags
    text = re.sub(r'#', '', text)
    text = re.sub(r'_', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r'\n', '', text)
    text = text.lower()
    return text


In [34]:
dfTreino['pre-processed'] = dfTreino['text'].apply(limpando_chars)
dfTreino = dfTreino[['text', 'pre-processed', 'sentiment']]
dfTreino.head()

,text,pre-processed,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...","mais uma vez, o sr. costner arrumou um filme p...",Negativo
1,Este é um exemplo do motivo pelo qual a maiori...,este é um exemplo do motivo pelo qual a maiori...,Negativo
2,"Primeiro de tudo eu odeio esses raps imbecis, ...","primeiro de tudo eu odeio esses raps imbecis, ...",Negativo
3,Nem mesmo os Beatles puderam escrever músicas ...,nem mesmo os beatles puderam escrever músicas ...,Negativo
4,Filmes de fotos de latão não é uma palavra apr...,filmes de fotos de latão não é uma palavra apr...,Negativo


### STOPWORDS

In [35]:
stopwords = nltk.corpus.stopwords.words('portuguese')

pontuacao = []
for ponto in punctuation:
    pontuacao.append(ponto)

stopwords = pontuacao = stopwords

### Tokenização e Stemmização

In [36]:
token_space = tokenize.WhitespaceTokenizer()

#### Função processar_texto()

In [37]:
def processar_texto(texto):
    stemmer = nltk.RSLPStemmer()
    stopwords = nltk.corpus.stopwords.words('portuguese')
    texto = re.sub(r'@[A-Za-z0-9]+', '', texto) # Removendo menções
    texto = re.sub(r'#\w+', '', texto) # Removendo #hashtags
    texto = re.sub(r'#', '', texto)
    texto = re.sub(r'_', '', texto)
    texto = re.sub(r'RT[\s]+', '', texto)
    texto = re.sub(r'https?:\/\/\S+', '', texto)
    texto = re.sub(r'\n', '', texto)
    pontuacao = []
    for ponto in punctuation:
        pontuacao.append(ponto)
    # Remove caracteres especiais
    texto = re.sub(r'[^\w\s]', '', texto)
    # Converte para minúsculas
    texto = texto.lower()
    texto = [stemmer.stem(palavra) for palavra in texto.split() if palavra not in stopwords]
    # Une acentos
    texto = [unidecode.unidecode(palavra) for palavra in texto]
    return ' '.join(texto)

In [39]:
dfTreino['text'][0]

'Mais uma vez, o Sr. Costner arrumou um filme por muito mais tempo do que o necessário. Além das terríveis seqüências de resgate no mar, das quais há muito poucas, eu simplesmente não me importei com nenhum dos personagens. A maioria de nós tem fantasmas no armário, e o personagem Costers é realizado logo no início, e depois esquecido até muito mais tarde, quando eu não me importava. O personagem com o qual deveríamos nos importar é muito arrogante e superconfiante, Ashton Kutcher. O problema é que ele sai como um garoto que pensa que é melhor do que qualquer outra pessoa ao seu redor e não mostra sinais de um armário desordenado. Seu único obstáculo parece estar vencendo Costner. Finalmente, quando estamos bem além do meio do caminho, Costner nos conta sobre os fantasmas dos Kutchers. Somos informados de por que Kutcher é levado a ser o melhor sem pressentimentos ou presságios anteriores. Nenhuma mágica aqui, era tudo que eu podia fazer para não desligar uma hora.'

In [40]:
print(processar_texto(dfTreino['text'][0]))

vez sr costn arrum film temp necess alem terr sequ resgat mar qual pouc simples import nenhum person maior fantasm armari person cost realiz log inici esquec tard import person dev import arrog superconfi ashton kutch problem sai garot pens melhor qualqu outr pesso redor mostr sinal armari desorden unic obstacul parec venc costn final bem alem mei caminh costn cont sobr fantasm kutch inform kutch lev melhor pressent pressagi anteri nenhum magic aqu tud pod faz deslig hor


In [41]:
dfTreino['pre-processed'] = dfTreino['text'].apply(processar_texto)

In [42]:
dfTreino.head()

,text,pre-processed,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",vez sr costn arrum film temp necess alem terr ...,Negativo
1,Este é um exemplo do motivo pelo qual a maiori...,exempl motiv maior film aca mesm gener chat na...,Negativo
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",prim tud odei rap imbecil pod agir arm pressio...,Negativo
3,Nem mesmo os Beatles puderam escrever músicas ...,beatl pud escrev music tod gost emb walt hill ...,Negativo
4,Filmes de fotos de latão não é uma palavra apr...,film fot lat palavr apropri verdad tant ous qu...,Negativo


In [43]:
dfTreino.text[2]

'Primeiro de tudo eu odeio esses raps imbecis, que não poderiam agir se tivessem uma arma pressionada contra suas testas. Tudo o que eles fazem é amaldiçoar e atirar um no outro e agir como uma versão clichê de gangsters. O filme não leva mais de cinco minutos para explicar o que está acontecendo antes que já estivessem no armazém. Não há um único personagem simpático nesse filme, com exceção do sem-teto, que também é o único com metade do cérebro. William Paxton e William Sadler são ambos "hill billies" e Sadler é tão vilão quanto os gângsteres. Eu não gostava dele desde o começo. O filme está cheio de violência sem sentido e especialidade de Walter Hills: pessoas caindo de janelas com vidros voando por toda parte. Não há praticamente nenhum enredo e é um grande problema quando você torce por ninguém. Todo mundo morre, exceto Paxton e o sem-teto e todos recebem o que merecem. Os dois únicos negros que podem atuar são o sem-teto e o viciado, mas são atores de profissão, não irritantes 

In [44]:
dfTreino['pre-processed'][2]

'prim tud odei rap imbecil pod agir arm pression contr test tud faz amaldico atir outr agir vers clich gangst film lev cinc minut explic acontec ant armazem unic person simpa ness film excec semtet unic metad cerebr will paxton will sadl ambos hill billi sadl tao vil quant gangst gost desd comec film chei viol sent especial walt hill pesso caind janel vidr vo tod part pratic nenhum enred grand problem torc ninguem tod mund morr excet paxton semtet tod receb merec doi unic negr pod atu semtet vici at profiss irrit rapp fei fiqu long dess porc observ 48 hor 1 2 vez diss min tem person gost sens hum nad alem at real elenc'

### Criando o Modelo

In [45]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Cria um vetorizador
tfidf = TfidfVectorizer()

# Transforma os textos em um vetor TF-IDF
vetor_tfidf = tfidf.fit_transform(dfTreino['pre-processed'])

# Divida os dados em um conjunto de treinamento e um conjunto de teste
treino, teste, classe_treino, classe_teste = train_test_split(vetor_tfidf, dfTreino['sentiment'], random_state=42)

# Cria um modelo de Naive Bayes
modelo = MultinomialNB(alpha=1.0)

# Treina o modelo
modelo.fit(treino, classe_treino)

# Teste o modelo
previsoes = modelo.predict(teste)

# Calcula a acurácia do modelo
acuracia = modelo.score(teste, classe_teste)

print(acuracia)


0.8506234015345269


In [46]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Calcula as métricas
acuracia = accuracy_score(classe_teste, previsoes)
f1_score = f1_score(classe_teste, previsoes, pos_label='Positivo')
precision_score = precision_score(classe_teste, previsoes, pos_label='Positivo')
recall_score = recall_score(classe_teste, previsoes, pos_label='Positivo')

# Cria um DataFrame com as métricas
metricas = pd.DataFrame({
    'Acurácia': acuracia,
    'F1-score': f1_score,
    'Precisão': precision_score,
    'Recall': recall_score
}, index=[''])

# Exibe o DataFrame
print(metricas)

  Acurácia  F1-score  Precisão    Recall
  0.850623  0.847987  0.856697  0.839452


In [47]:
from sklearn.metrics import classification_report, roc_auc_score

report = classification_report(classe_teste, previsoes)
print(report)


              precision    recall  f1-score   support

    Negativo       0.84      0.86      0.85      6302
    Positivo       0.86      0.84      0.85      6210

    accuracy                           0.85     12512
   macro avg       0.85      0.85      0.85     12512
weighted avg       0.85      0.85      0.85     12512



### Otimização

In [48]:
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import MultinomialNB

param_grid = {
    'alpha': [0.1, 0.5, 1.0],
}

grid_search = GridSearchCV(estimator=MultinomialNB(), param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(treino, classe_treino)

best_params = grid_search.best_params_
print("Melhores hiperparâmetros:", best_params)


best_model = grid_search.best_estimator_
y_pred = best_model.predict(teste)
accuracy = accuracy_score(classe_teste, y_pred)
print("Acurácia do modelo otimizado:", accuracy)


Melhores hiperparâmetros: {'alpha': 1.0}
Acurácia do modelo otimizado: 0.8506234015345269


### Cross Validation

In [49]:
from sklearn.model_selection import cross_val_score, KFold

# Defina o número de folds (k)
num_folds = 5



# Crie um objeto KFold para controlar a divisão dos dados
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Realize a validação cruzada
scores = cross_val_score(modelo, vetor_tfidf , dfTreino['sentiment'], cv=kf, scoring='accuracy')
mean_accuracy = scores.mean()
std_accuracy = scores.std()
print(f'Acurácia média: {mean_accuracy:.2f}')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}')


Acurácia média: 0.85
Desvio padrão da acurácia: 0.00


# Predições

### Função predict_it

In [50]:
def predict_it(frase):
    # Aplica a função de pré-processamento
    texto_processado = processar_texto(frase)
    # Transforma o texto em um vetor TF-IDF
    vetor_tfidf = tfidf.transform([texto_processado])
    # Prevê a classificação da frase
    previsao = modelo.predict(vetor_tfidf)
    probabilidades = modelo.predict_proba(vetor_tfidf).astype('float')[0]
    pos = probabilidades[1]
    neg = probabilidades[0]
    if pos < 0.51 and neg < 0.51:
        return 'Neutro'
    else:
        return previsao.astype('str')[0]

In [51]:
frase = 'ele foi muito bem'
print(predict_it(frase))


Positivo


## Salvando o modelo.

In [52]:
import joblib

joblib.dump(modelo, './model/nb_predictor.joblib')

['./model/nb_predictor.joblib']

In [53]:
import pickle
pickle.dump(tfidf.vocabulary_,open("./model/tfidf.pkl","wb"))